## Predictive Modeling: Using Regression models to predict price of used cars.
This project aims to show the process of creating Machine Learning models based on Simple, Multiple and Polynomial Regressions.
In this project, R2 is used to evaluate the performance of the models. Pearson Coeficient's P-Value of <= 0.05 rejects the null hypotesis.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output

In [2]:
def ETL(pathfile):
    df = pd.read_csv(pathfile)
    df['car_name'] = df['manufacturer_name'] + '-' + df['model_name']
    df = df[df['price_usd'] > 0]
    lista = []                                              #
    for car in list(set(df['car_name'])):                   #
        if len(df[df['car_name'] == car]) < 20:             #
            lista.append(car)                               # Removing cars that appear less than 20 times.
    indexes = df[df['car_name'].isin(lista)].index          #
    df = df.drop(index=indexes)                             #
    return df.drop(['feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','duration_listed','engine_has_gas','engine_type','has_warranty','is_exchangeable','location_region','number_of_photos','body_type','up_counter','feature_0','manufacturer_name','model_name'],axis=1)

#ETL e Data Cleaning
def PrepData(df,carname):
    df = df[df['car_name'] == carname]
    for column in df.columns:
        if (df[column].values == df[column].values[0]).all():
            df = df.drop(columns=column)
    TargetVar = df['price_usd']
    df = df.drop(columns=['price_usd'],axis=1)
    df.insert(0,'price_usd',TargetVar)
    dfobject = df.select_dtypes(include='object')
    df = df.select_dtypes(exclude='object')
    dummiesdf = pd.get_dummies(dfobject)
    mergeddf = pd.merge(df,dummiesdf,left_index=True,right_index=True)
    targetcol = mergeddf['price_usd']                                                            
    return mergeddf, dfobject, targetcol

#Analysis Function
def PlotandAnalysis(df,BPdfobj,targetcol,polydegrees):
    def BoxplotObjects(BPdfobj,targetcol): ## Show counts on plot ##better show car colors
        fig, axs = plt.subplots(ncols=len(list(BPdfobj)), figsize=(16,4),sharey=True)
        t = 0
        for column in list(BPdfobj):
            sns.boxplot(x=BPdfobj[column],y=targetcol,data=BPdfobj, ax=axs[t])
            t += 1
        plt.show()
        plt.close()

    #Simple Linear Regression Model and Plot
    def SLRModelandPlot(df,targetcol):
        x = df[[NLargAbsCoef(df,1)[0]]].values
        m = LinearRegression().fit(x,targetcol)
        fig, axs = plt.subplots(ncols=2, figsize=(16,4))
        ax0 = sns.regplot(x=x,y=targetcol,data=df,ax=axs[0])
        ax1 = sns.residplot(x=x,y=targetcol,data=df,ax=axs[1])
        ax0.set_ylim(0,)
        ax0.set_xlim(x.min(),)
        plt.show()
        plt.close()
        print('Simple Linear Regression Model Equation:\n',m.intercept_,'+',float(m.coef_),'*',targetcol.name)
        return x,m

    #Multiple Linear Regression Model and Plot
    def MLRModelandPlot(df,targetcol,): #adapt to get max num of var and better show model equation
        x = df[AllCoef(df)].values
        m = LinearRegression().fit(x,targetcol)
        plt.figure(figsize=(16,4))
        ax1 = sns.kdeplot(targetcol, color="r", label="Actual Value")
        sns.kdeplot(m.predict(x), color="b", label="Predicted Values", ax=ax1)
        plt.title('Actual (red) vs Predicted (blue) Price')
        plt.xlabel('Price')
        plt.ylabel('Proportion of Cars')
        plt.xlim(0,)
        plt.show()
        plt.close()
        print("MLR Intercept:",m.intercept_)
        print("MLR Coefs:",list(m.coef_))
        return x,m

    #Polynomial Regression Model and Plot
    def PlotPR(df,targetcol,polydegrees): #adapt to get best degree
        x = df[NLargAbsCoef(df,1)[0]].values
        m = np.poly1d(np.polyfit(x, targetcol, polydegrees))
        x_new = np.linspace(x.min(),x.max())
        y_new = m(x_new)
        plt.plot(x, targetcol, '.', x_new, y_new, '-')
        ax = plt.gca()
        ax.set_facecolor((0.898, 0.898, 0.898))
        fig = plt.gcf()
        plt.show()
        plt.close()
        print('Polynomial Regression Model Equation:\n',m)
        return x,m

    #R2 calculation
    def R2(realY,predictY):
        return r2_score(realY,predictY)

    #Comparison of best model by R2 criteria
    def ModelComparison(x1,m1,x2,m2,x3,m3,targetcol):
        print('\n-------------------------------------------------------------------\n')
        md = ['Simple Linear Regression','Multiple Linear Regression','Polynomial Regression']
        preds = [m1.predict(x1),m2.predict(x2),m3(x3)]
        bestr2 = 0
        for pred, q in zip(list(preds), md):
            r2 = R2(targetcol,pred)
            print(q,'R2:',r2)
            if r2 > bestr2:
                bestr2 = r2
                bestq = q
        print('\nBest model by R2 is',bestq,'with R2 of',bestr2)

    BoxplotObjects(BPdfobj,targetcol)
    slrx, slrm = SLRModelandPlot(df,targetcol)
    mlrx, mlrm = MLRModelandPlot(df,targetcol)
    prx, prm = PlotPR(df,targetcol,polydegrees)
    ModelComparison(slrx,slrm,mlrx,mlrm,prx,prm,targetcol)

In [3]:
#Pearson coefficient calculation
def CoefPearson(df):
    lcoef = []
    lp = []
    for col in list(df.columns):
        coef, p = stats.pearsonr(df[col],df['price_usd'])
        lcoef.append(coef)
        lp.append(round(p,3))
    df = pd.DataFrame(lcoef).merge(pd.DataFrame(lp),on=df.columns).rename(columns={'key_0':'IndependentVar','0_x':'PearsonCoef','0_y':'P-Value'}).drop([0])
    return df[df['P-Value'] <= 0.05].reset_index(drop=True)

#Listing n independent variables with p <= 0.05
def NLargAbsCoef(df,n):
    df = CoefPearson(df)
    df = df.loc[df['PearsonCoef'].abs().nlargest(1,keep='first').index]
    return list(df['IndependentVar'])

#Listing all indepedent variables with p <= 0.05
def AllCoef(df):
    df = CoefPearson(df)
    return list(df['IndependentVar'])

In [4]:
def car_selector(pathfile):
    data = ETL(pathfile)
    dropdown_manufacturer = widgets.Dropdown(options = sorted(list(set(data['car_name']))))
    output = widgets.Output()

    def output_by_manufacturer(df,carname):
        with output:
            IPython.display.clear_output(wait=True)
            display(PlotandAnalysis(*PrepData(df,carname),4))     

    def dropdown_state_eventhandler(change):
            display(input_widgets)
            output_by_manufacturer(data,change.new) 

    dropdown_manufacturer.observe(dropdown_state_eventhandler, names='value')
    input_widgets = widgets.HBox([dropdown_manufacturer])
    display(input_widgets,output)

In [5]:
car_selector('../PredictiveModelingRegression/cars.csv')

Output()